---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

## `test.csv`

Examine `test.csv`

In [2]:
train_raw = pd.read_csv('train.csv', engine='python')
train_raw.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [3]:
train_raw.dtypes

ticket_id                       int64
agency_name                    object
inspector_name                 object
violator_name                  object
violation_street_number       float64
violation_street_name          object
violation_zip_code            float64
mailing_address_str_number    float64
mailing_address_str_name       object
city                           object
state                          object
zip_code                       object
non_us_str_code                object
country                        object
ticket_issued_date             object
hearing_date                   object
violation_code                 object
violation_description          object
disposition                    object
fine_amount                   float64
admin_fee                     float64
state_fee                     float64
late_fee                      float64
discount_amount               float64
clean_up_cost                 float64
judgment_amount               float64
payment_amou

Make a clone copy so that it's easier to get a fresh copy (without pd.read_csv() again) in case I mess up something

In [4]:
train = train_raw.copy()

Delete columns that won't be available in `train.csv`

In [5]:
remove = ['payment_amount',
          'payment_date',
          'payment_status',
          'balance_due',
          'collection_status',
          'compliance_detail']
for column in remove:
    del train[column]
train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ticket_id                   250306 non-null  int64  
 1   agency_name                 250306 non-null  object 
 2   inspector_name              250306 non-null  object 
 3   violator_name               250272 non-null  object 
 4   violation_street_number     250306 non-null  float64
 5   violation_street_name       250306 non-null  object 
 6   violation_zip_code          0 non-null       float64
 7   mailing_address_str_number  246704 non-null  float64
 8   mailing_address_str_name    250302 non-null  object 
 9   city                        250306 non-null  object 
 10  state                       250213 non-null  object 
 11  zip_code                    250305 non-null  object 
 12  non_us_str_code             3 non-null       object 
 13  country       

In [7]:
train.nunique()

ticket_id                     250306
agency_name                        5
inspector_name                   173
violator_name                 119992
violation_street_number        19175
violation_street_name           1791
violation_zip_code                 0
mailing_address_str_number     15826
mailing_address_str_name       37896
city                            5184
state                             59
zip_code                        4251
non_us_str_code                    2
country                            5
ticket_issued_date             86979
hearing_date                    6222
violation_code                   235
violation_description            258
disposition                        9
fine_amount                       43
admin_fee                          2
state_fee                          2
late_fee                          37
discount_amount                   13
clean_up_cost                      1
judgment_amount                   57
grafitti_status                    1
c

## Features selection

Select features for training. Columns that are not selected because they don't have connection to target value or may be a source of data leakage (e.g. `ticket_id`). Some features are not included because they have too many unique values (e.g. `city` unique values are half of the total entries). Other excluded columns have too many `NaN` values (e.g. `violation_zip_code`)

In [8]:
train = train.loc[:, ['agency_name', 'inspector_name', 'state',
                     'country', 'ticket_issued_date', 'hearing_date', 'disposition',
                      'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
                     'discount_amount', 'judgment_amount', 'compliance']]
train.head()

,agency_name,inspector_name,state,country,ticket_issued_date,hearing_date,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance
0,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",IL,USA,2004-03-16 11:40:00,2005-03-21 10:30:00,Responsible by Default,250.0,20.0,10.0,25.0,0.0,305.0,0.0
1,"Buildings, Safety Engineering & Env Department","Williams, Darrin",MI,USA,2004-04-23 12:30:00,2005-05-06 13:30:00,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,855.0,1.0
2,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",MI,USA,2004-04-26 13:40:00,2005-03-29 10:30:00,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,NaN
3,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",MI,USA,2004-04-26 13:30:00,NaN,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,NaN
4,"Buildings, Safety Engineering & Env Department","Sims, Martinzie",MI,USA,2004-04-26 13:00:00,2005-03-29 10:30:00,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,NaN


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   agency_name         250306 non-null  object 
 1   inspector_name      250306 non-null  object 
 2   state               250213 non-null  object 
 3   country             250306 non-null  object 
 4   ticket_issued_date  250306 non-null  object 
 5   hearing_date        237815 non-null  object 
 6   disposition         250306 non-null  object 
 7   fine_amount         250305 non-null  float64
 8   admin_fee           250306 non-null  float64
 9   state_fee           250306 non-null  float64
 10  late_fee            250306 non-null  float64
 11  discount_amount     250306 non-null  float64
 12  judgment_amount     250306 non-null  float64
 13  compliance          159880 non-null  float64
dtypes: float64(7), object(7)
memory usage: 26.7+ MB


## Data cleaning

Drop rows with empty fields (I decided not to fill missing values during training)

In [10]:
train = train.dropna()
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159569 entries, 0 to 250293
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   agency_name         159569 non-null  object 
 1   inspector_name      159569 non-null  object 
 2   state               159569 non-null  object 
 3   country             159569 non-null  object 
 4   ticket_issued_date  159569 non-null  object 
 5   hearing_date        159569 non-null  object 
 6   disposition         159569 non-null  object 
 7   fine_amount         159569 non-null  float64
 8   admin_fee           159569 non-null  float64
 9   state_fee           159569 non-null  float64
 10  late_fee            159569 non-null  float64
 11  discount_amount     159569 non-null  float64
 12  judgment_amount     159569 non-null  float64
 13  compliance          159569 non-null  float64
dtypes: float64(7), object(7)
memory usage: 18.3+ MB


Process string features. Steps include:
- Convert all to lowercase
- Use `sklearn.preprocessing.LabelEncoder` to encode string values to integer values

In [11]:
string_columns = ['agency_name', 'inspector_name', 'state', 'country', 'disposition']

In [12]:
for column in string_columns:
    train[column] = train[column].str.lower()
train.head()

,agency_name,inspector_name,state,country,ticket_issued_date,hearing_date,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance
0,"buildings, safety engineering & env department","sims, martinzie",il,usa,2004-03-16 11:40:00,2005-03-21 10:30:00,responsible by default,250.0,20.0,10.0,25.0,0.0,305.0,0.0
1,"buildings, safety engineering & env department","williams, darrin",mi,usa,2004-04-23 12:30:00,2005-05-06 13:30:00,responsible by determination,750.0,20.0,10.0,75.0,0.0,855.0,1.0
5,"buildings, safety engineering & env department","sims, martinzie",ca,usa,2004-05-01 11:50:00,2005-03-21 10:30:00,responsible by default,250.0,20.0,10.0,25.0,0.0,305.0,0.0
6,"buildings, safety engineering & env department","williams, darrin",mi,usa,2004-06-14 14:15:00,2005-02-22 15:00:00,responsible by default,750.0,20.0,10.0,75.0,0.0,855.0,0.0
7,"buildings, safety engineering & env department","williams, darrin",mi,usa,2004-06-16 12:30:00,2005-02-22 15:00:00,responsible by default,100.0,20.0,10.0,10.0,0.0,140.0,0.0


In [13]:
for column in string_columns:
    num = train[column].nunique()
    print('{}: {}'.format(column, num))
    if num > 4:
        print(train[column].unique()[:4])
    else:
        print(train[column].unique())
    print()

agency_name: 5
['buildings, safety engineering & env department' 'health department'
 'department of public works' 'detroit police department']

inspector_name: 158
['sims, martinzie' 'williams, darrin' 'jones, leah' 'gailes, orbie j']

state: 59
['il' 'mi' 'ca' 'ny']

country: 4
['usa' 'cana' 'aust' 'germ']

disposition: 4
['responsible by default' 'responsible by determination'
 'responsible by admission' 'responsible (fine waived) by deter']



In [14]:
from sklearn.preprocessing import LabelEncoder
str_dict = {}
for column in string_columns:
    str_dict[column] = LabelEncoder()
    str_dict[column].fit(train[column])
    train[column] = str_dict[column].transform(train[column])

train.head()

,agency_name,inspector_name,state,country,ticket_issued_date,hearing_date,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance
0,0,124,16,3,2004-03-16 11:40:00,2005-03-21 10:30:00,2,250.0,20.0,10.0,25.0,0.0,305.0,0.0
1,0,152,24,3,2004-04-23 12:30:00,2005-05-06 13:30:00,3,750.0,20.0,10.0,75.0,0.0,855.0,1.0
5,0,124,6,3,2004-05-01 11:50:00,2005-03-21 10:30:00,2,250.0,20.0,10.0,25.0,0.0,305.0,0.0
6,0,152,24,3,2004-06-14 14:15:00,2005-02-22 15:00:00,2,750.0,20.0,10.0,75.0,0.0,855.0,0.0
7,0,152,24,3,2004-06-16 12:30:00,2005-02-22 15:00:00,2,100.0,20.0,10.0,10.0,0.0,140.0,0.0


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159569 entries, 0 to 250293
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   agency_name         159569 non-null  int32  
 1   inspector_name      159569 non-null  int32  
 2   state               159569 non-null  int32  
 3   country             159569 non-null  int32  
 4   ticket_issued_date  159569 non-null  object 
 5   hearing_date        159569 non-null  object 
 6   disposition         159569 non-null  int32  
 7   fine_amount         159569 non-null  float64
 8   admin_fee           159569 non-null  float64
 9   state_fee           159569 non-null  float64
 10  late_fee            159569 non-null  float64
 11  discount_amount     159569 non-null  float64
 12  judgment_amount     159569 non-null  float64
 13  compliance          159569 non-null  float64
dtypes: float64(7), int32(5), object(2)
memory usage: 15.2+ MB


Process date values. It can be said that `time_difference` is an engineered feature. `ticket_issued_date` and `hearing_date` are removed

In [16]:
train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'])
train['hearing_date'] = pd.to_datetime(train['hearing_date'])
train['time_difference'] = train['hearing_date'] - train['ticket_issued_date']
train.head()

,agency_name,inspector_name,state,country,ticket_issued_date,hearing_date,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,time_difference
0,0,124,16,3,2004-03-16 11:40:00,2005-03-21 10:30:00,2,250.0,20.0,10.0,25.0,0.0,305.0,0.0,369 days 22:50:00
1,0,152,24,3,2004-04-23 12:30:00,2005-05-06 13:30:00,3,750.0,20.0,10.0,75.0,0.0,855.0,1.0,378 days 01:00:00
5,0,124,6,3,2004-05-01 11:50:00,2005-03-21 10:30:00,2,250.0,20.0,10.0,25.0,0.0,305.0,0.0,323 days 22:40:00
6,0,152,24,3,2004-06-14 14:15:00,2005-02-22 15:00:00,2,750.0,20.0,10.0,75.0,0.0,855.0,0.0,253 days 00:45:00
7,0,152,24,3,2004-06-16 12:30:00,2005-02-22 15:00:00,2,100.0,20.0,10.0,10.0,0.0,140.0,0.0,251 days 02:30:00


In [17]:
train['time_difference'] = train['time_difference'].dt.days
del train['ticket_issued_date']
del train['hearing_date']
train.head()

,agency_name,inspector_name,state,country,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance,time_difference
0,0,124,16,3,2,250.0,20.0,10.0,25.0,0.0,305.0,0.0,369
1,0,152,24,3,3,750.0,20.0,10.0,75.0,0.0,855.0,1.0,378
5,0,124,6,3,2,250.0,20.0,10.0,25.0,0.0,305.0,0.0,323
6,0,152,24,3,2,750.0,20.0,10.0,75.0,0.0,855.0,0.0,253
7,0,152,24,3,2,100.0,20.0,10.0,10.0,0.0,140.0,0.0,251


In [18]:
train.nunique()

agency_name          5
inspector_name     158
state               59
country              4
disposition          4
fine_amount         40
admin_fee            1
state_fee            1
late_fee            37
discount_amount     12
judgment_amount     57
compliance           2
time_difference    582
dtype: int64

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159569 entries, 0 to 250293
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   agency_name      159569 non-null  int32  
 1   inspector_name   159569 non-null  int32  
 2   state            159569 non-null  int32  
 3   country          159569 non-null  int32  
 4   disposition      159569 non-null  int32  
 5   fine_amount      159569 non-null  float64
 6   admin_fee        159569 non-null  float64
 7   state_fee        159569 non-null  float64
 8   late_fee         159569 non-null  float64
 9   discount_amount  159569 non-null  float64
 10  judgment_amount  159569 non-null  float64
 11  compliance       159569 non-null  float64
 12  time_difference  159569 non-null  int64  
dtypes: float64(7), int32(5), int64(1)
memory usage: 14.0 MB


## Training an ML model

Model chosen: Random Forest from `sklearn.ensemble.RandomForestClassifier`

Use `sklearn.model_selection.GridSearchCV` to select the best parameters. Tuning parameters include number of trees `n_estimators` and number of subset features to be considered in each tree `max_features`

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(RandomForestClassifier(), param_grid={'n_estimators': [10, 30, 100], 'max_features': [1, 2, 4]}, scoring='roc_auc')

In [21]:
X_train = train.iloc[:,:-2]
X_train['time_difference'] = train.iloc[:,-1]
y_train = train.iloc[:,-2]
X_train.head()

,agency_name,inspector_name,state,country,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,time_difference
0,0,124,16,3,2,250.0,20.0,10.0,25.0,0.0,305.0,369
1,0,152,24,3,3,750.0,20.0,10.0,75.0,0.0,855.0,378
5,0,124,6,3,2,250.0,20.0,10.0,25.0,0.0,305.0,323
6,0,152,24,3,2,750.0,20.0,10.0,75.0,0.0,855.0,253
7,0,152,24,3,2,100.0,20.0,10.0,10.0,0.0,140.0,251


In [22]:
grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

## `test.csv`

Import and prepare `test.csv` exactly the same as how `train.csv` was processed. Use the same transformation

In [23]:
test_raw = pd.read_csv('test.csv')
test_raw.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [24]:
test = test_raw.copy()

In [25]:
test = test.loc[:, ['agency_name', 'inspector_name', 'state',
                     'country', 'ticket_issued_date', 'hearing_date', 'disposition',
                      'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
                     'discount_amount', 'judgment_amount']]
test = test.fillna(0)
for column in string_columns:
    test[column] = test[column].str.lower()
test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])
test['hearing_date'] = pd.to_datetime(test['hearing_date'])
test['time_difference'] = test['hearing_date'] - test['ticket_issued_date']
test['time_difference'] = test['time_difference'].dt.days
del test['ticket_issued_date']
del test['hearing_date']


for column in string_columns:
    le_dict = dict(zip(str_dict[column].classes_, str_dict[column].transform(str_dict[column].classes_)))
    test[column] = test[column].apply(lambda x: le_dict.get(x, -1))

test.head()

,agency_name,inspector_name,state,country,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,time_difference
0,1,51,24,3,2,200.0,20.0,10.0,20.0,0.0,250.0,14
1,1,74,24,3,2,1000.0,20.0,10.0,100.0,0.0,1130.0,31
2,1,74,24,3,2,100.0,20.0,10.0,10.0,0.0,140.0,31
3,1,137,24,3,2,200.0,20.0,10.0,20.0,0.0,250.0,32
4,1,137,24,3,2,100.0,20.0,10.0,10.0,0.0,140.0,39


In [26]:
output = pd.DataFrame(grid.predict_proba(test))
output = pd.merge(output[1], test_raw['ticket_id'], left_index=True, right_index=True)
output = output.set_index('ticket_id')
output = output[1]
output.head()

ticket_id
284932    0.029104
285362    0.000000
285361    0.222343
285338    0.580000
285346    0.054595
Name: 1, dtype: float64

## Final output

In [27]:
output.head()

ticket_id
284932    0.029104
285362    0.000000
285361    0.222343
285338    0.580000
285346    0.054595
Name: 1, dtype: float64

## Coursera submission

Function for Coursera submission

In [28]:
def blight_model():
    
    # Your code here
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.preprocessing import LabelEncoder


    train = pd.read_csv('train.csv', engine='python')
    test = pd.read_csv('test.csv')

    train = train.loc[:, ['agency_name', 'inspector_name', 'state',
                                  'country', 'ticket_issued_date', 'hearing_date', 'disposition',
                                  'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
                                  'discount_amount', 'judgment_amount', 'compliance']]
    train = train.dropna()
    string_columns = ['agency_name', 'inspector_name', 'state', 'country', 'disposition']
    for column in string_columns:
        train[column] = train[column].str.lower()
    train['ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'])
    train['hearing_date'] = pd.to_datetime(train['hearing_date'])
    train['time_difference'] = train['hearing_date'] - train['ticket_issued_date']
    train['time_difference'] = train['time_difference'].dt.days
    del train['ticket_issued_date']
    del train['hearing_date']

    str_dict = {}
    for column in string_columns:
        str_dict[column] = LabelEncoder()
        train[column] = str_dict[column].fit_transform(train[column])

    grid = GridSearchCV(RandomForestClassifier(n_estimators=50, n_jobs=4), param_grid={'max_depth': [1, 3, 10], 'max_features': [1, 2, 4]}, scoring='roc_auc')
    X_train = train.iloc[:,:-2]
    X_train['time_difference'] = train.iloc[:,-1]
    y_train = train.iloc[:,-2]
    # print(train.head())
    grid.fit(X_train, y_train)

    ticket = test['ticket_id']
    test = test.loc[:, ['agency_name', 'inspector_name', 'state',
                        'country', 'ticket_issued_date', 'hearing_date', 'disposition',
                        'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
                        'discount_amount', 'judgment_amount']]
    test = test.fillna(0)
    for column in string_columns:
        test[column] = test[column].str.lower()
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])
    test['hearing_date'] = pd.to_datetime(test['hearing_date'])
    test['time_difference'] = test['hearing_date'] - test['ticket_issued_date']
    test['time_difference'] = test['time_difference'].dt.days
    del test['ticket_issued_date']
    del test['hearing_date']


    for column in string_columns:
        le_dict = dict(zip(str_dict[column].classes_, str_dict[column].transform(str_dict[column].classes_)))
        test[column] = test[column].apply(lambda x: le_dict.get(x, -1))

    output = pd.DataFrame(grid.predict_proba(test))
    output = pd.merge(output, pd.DataFrame(ticket), left_index=True, right_index=True)
    output = output.set_index('ticket_id')
    output.columns = ['non-compliance', 'compliance']

    return output['compliance']

In [29]:
blight_model()

ticket_id
284932    0.058132
285362    0.017173
285361    0.072795
285338    0.062510
285346    0.069651
            ...   
376496    0.010917
376497    0.010917
376499    0.070381
376500    0.070381
369851    0.497054
Name: compliance, Length: 61001, dtype: float64